In [82]:
import pandas as pd
import numpy as np
import spacy
import warnings
from corus import load_rudrec
warnings.filterwarnings("ignore")

In [83]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Load data 

In [113]:
drugs = list(load_rudrec('data/rudrec_annotated.json'))

In [91]:
caws_df = pd.read_csv('data/train/train.csv')

In [92]:
caws_labels = pd.read_json('data/train/train_labels.json').T

In [93]:
hand_labeled_caws_df = pd.read_json('data/labeled_by_hand_version_2.json')

hand_labeled_caws_df['id'] = hand_labeled_caws_df['id'] - np.ones(len(hand_labeled_caws_df['id']))
hand_labeled_caws_df['id'] = hand_labeled_caws_df['id'].astype(int)

hand_labeled_caws_df = hand_labeled_caws_df.drop([0, 239])

#### Simple visualization 

In [94]:
def render_annotations(index):
    colors = {"Annotation" :"linear-gradient(90deg, #aa9cfc, #fc9ce7)" } 
    options = {"colors": colors}
    
    text = labeled_by_hand_df['data'].iloc[index]['text']
    annotations = labeled_by_hand_df['annotations'].iloc[index][0]['result']
    
    ents = []
    for annotation in annotations:
        annotation = annotation['value']
        
        ents.append({
            'start': annotation['start'],
            'end': annotation['end'],
            'label': "Symptom"
        })
    
    doc = {'text': text, 'ents': ents}
    
    spacy.displacy.render(doc, style="ent", options = options , manual=True)

In [102]:
render_annotations(0)

#### Collate data 

In [103]:
from data_collator import process_rudrec, process_caws

In [106]:
rudrec_data = process_rudrec(drugs)

In [108]:
caws_data = process_caws(caws_df, caws_labels, hand_labeled_caws_df)

In [110]:
medical_data = rudrec_data + caws_data

In [111]:
medical_df = pd.DataFrame(medical_data)

In [112]:
medical_df

,tokens,tags
0,"[нам, прописали, ,, так, мой, ребенок, сыпью, ...","[O, O, O, O, O, O, B-ADR, I-ADR, O, B-ADR, I-A..."
1,"[Пила, этот, препарат, для, повышения, иммунит...","[O, O, O, O, B-DI, I-DI, O, O, O, O]"
2,"[В, месяц, по, нескольку, раз, причем, со, все...","[O, O, O, O, O, O, O, O, O, O, O, B-DI, O, B-D..."
3,"[Хочу, с, Вами, поделиться, отзывом, о, средст...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,"[Как, только, мой, муж, видит, признаки, прост...","[O, O, O, O, O, O, B-DI, O, O, O, O, O, O, O, O]"
...,...,...
1567,"[Скажите, пожалуйста, ,, что, делать, если, те...","[O, O, O, O, O, O, O, O, O, O, O, B-DI, I-DI, ..."
1568,"[Корова, не, встает, на, задние, ноги, ., Что,...","[O, B-DI, I-DI, I-DI, I-DI, I-DI, O, O, O, O]"
1569,"[У, телят, текут, белые, сопли, ,, что, это, т...","[O, O, O, B-DI, I-DI, O, O, O, O, O, O, O, O]"
1570,"[У, быка, заболели, ноги, ,, как-то, так, резк...","[O, O, B-DI, I-DI, O, O, O, O, O, O, O, O, O, ..."
